<a href="https://colab.research.google.com/github/awesomeplaya211/Miscellaneous/blob/master/Selfishness_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
from collections import deque

In [ ]:
# Im gonna rewrite this, so I will do it for the new version


number_selfish = 25 # stores the number of selfish people
number_hive = 25 
number_food = 50

def sim_one():
    global number_selfish, number_hive, number_food
    selfish_food = [0 for i in range(number_selfish)]
    hive_food = 0
    food_pos = [[] for i in range(number_food)]

    for i in range(number_selfish):
        food_pos[random.randint(0, number_food - 1)].append([1, i])

    for i in range(number_hive):
        food_pos[random.randint(0, number_food - 1)].append([0, 0])
    #print(food_pos)
    for food in food_pos:
        if len(food) == 0 or sum([x[0] for x in food]) > 1:
            continue
        if sum([x[0] for x in food]) == 1 and len(food) == 1:
            #print('selfish', food)
            selfish_food[food[[x[0] for x in food].index(1)][1]] += 2
        elif sum([x[0] for x in food]) == 1:
            selfish_food[food[[x[0] for x in food].index(1)][1]] += 1
        else:
            #print('hive', food)
            hive_food += 2
    for i in selfish_food:
        if i == 2:
            number_selfish += 1
        elif i == 0:
            number_selfish -= 1
    number_hive = hive_food
    #print(number_hive)

for i in range(100):
    sim_one()
print(number_selfish, number_hive)


24 0


In [ ]:
#This is the real code

class Message:
    def __init__(self, node, direction, works = False):
        self.idx = node # stores the point on this map that this message references
        self.direction = direction # stores the direction that this message refences 
        '''
        0 - up
        1 - right
        2 - down
        3 - left
        '''
        self.works = works # if works is true, then if you move in the indicated direction from the point, you will make it to the ending 

class Node:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y

class Graph:
    def __init__(self):
        self.adj_list = 0
        self.visited = 0

class Bot:
    def __init__(self, node, botnet, selfish = False):
        self.node = node # stores starting node
        self.botnet = botnet # stores botnet for message sending
        self.selfish = selfish # stores whether or not the bot is selfish (whether or not it communicates with its peers)
        self.messages = []
        self.min_length = -1
        self.min_instructions = []
    def message(self, m):
        self.messages.append(m)
    def move(self, mov, node):    
        if node.name == 'b':
            return mov
        moves = mov
        self.botnet.graph.visited[node] = True
        for n in self.botnet.graph.adj_list[node]:
            moved = False
            if not self.botnet.graph.visited[n[0]]:
                moves.append(self.botnet.graph.adj_list[n[0]][[[x[0].x, x[0].y] for x in self.botnet.graph.adj_list[n[0]]].index([node.x, node.y])])
                moved = True
                return self.move(moves, n[0])
            if not moved and not self.selfish:
                self.messages.append(Message(n[0], n[1], False))
        return False

        

    
class MazeBotNet:
    def __init__(self, maze, selfish = False): # Initialize a botnet
        self.selfish = selfish
        self.maze = self.convert_to_array(maze)
        self.start = [0, 0]
        self.finish = [0, 0]
        self.starts()
        self.bots = [] # Store the array of bots that the botnet controls
        self.graph = self.maze_to_graph()
        self.nodes = 0
        #self.curr_start = 0
        #self.curr_node = 0
    def solve(self):
        new_maze = self.maze[:]
        for i in range(len(self.bots)):
            moves = self.bots[i].move([], self.bots[i].node)
            if moves != False:
                for x in moves:
                    new_maze[x[0].x][x[0].y] = str(x[1])
                new_maze[self.finish[0]][self.finish[1]] = 'b'
                print('\n'.join([''.join(s) for s in new_maze]))
            else:
                print('No Solution')
    def convert_to_array(self, maze):
        m = []
        for s in maze:
            m.append(list(s))
        return m
    def message(self, message):
        for bot in self.bots:
            bot.message(message)
    def starts(self):
        for i in range(len(self.maze)):
            for j in range(len(self.maze[i])):
                if self.maze[i][j] == 'a':
                    self.start = [i, j]
                if self.maze[i][j] == 'b':
                    self.finish = [i, j]
        #print(self.start, self.finish)
    def neighboring_nodes(self, new_maze, node):
        x, y = node.x, node.y
        closest_nodes = [None, None, None, None]
        a, b = x + 1, y
        while a < len(self.maze):
            if new_maze[a][b] == '#':
                break
            if new_maze[a][b] == 'n':
                closest_nodes[0] = [a, b]
            a += 1
        a, b = x - 1, y
        while a >= 0:
            if new_maze[a][b] == '#':
                break
            if new_maze[a][b] == 'n':
                closest_nodes[1] = [a, b]
            a -= 1
        a, b = x, y + 1
        while b < len(self.maze[0]):
            if new_maze[a][b] == '#':
                break
            if new_maze[a][b] == 'n':
                closest_nodes[2] = [a, b]
            b += 1
        a, b = x, y - 1
        while b >= 0:
            if new_maze[a][b] == '#':
                break
            if new_maze[a][b] == 'n':
                closest_nodes[3] = [a, b]
            b -= 1
        return closest_nodes 
    def maze_to_graph(self):
        new_maze = self.maze[:]
        size = len(new_maze)
        nodes = []
        graph = Graph()
        for i in range(len(new_maze)):
            for j in range(len(new_maze[i])):
                if new_maze[i][j] == '#':
                    continue
                order = 0
                found_a, found_b = False, False
                if new_maze[i][j] == 'b' or new_maze[i][j] == 'a':
                    order += 2
                try:
                    if i == size:
                        raise IndexError()
                    if new_maze[i + 1][j] != '#':
                        found_a = True
                        order += 1
                except IndexError:
                    pass
                try:
                    if i == 0:
                        raise IndexError()
                    if new_maze[i - 1][j] != '#' and not found_a:
                        order += 1
                except IndexError:
                    pass
                try:
                    if j == len(new_maze[0]):
                        raise IndexError()
                    if new_maze[i][j + 1] != '#':
                        found_b = True
                        order += 1
                except IndexError:
                    pass
                try:
                    if j == 0:
                        raise IndexError()
                    if new_maze[i][j - 1] != '#' and not found_b:
                        order += 1
                except IndexError:
                    pass
                if order > 1:
                    nodes.append(Node(new_maze[i][j], i, j))
        nodes.sort(key=lambda node: node.y)
        self.nodes = nodes
        for node in nodes:
            self.bots.append(Bot(node, self, self.selfish))
            new_maze[node.x][node.y] = 'n'
        graph.adj_list = {
            node : [] for node in nodes
        }
        graph.visited = {
            node : False for node in nodes
        }
        for x in range(len(nodes)):
            ls = self.neighboring_nodes(new_maze, nodes[x])
            #print(ls)
            for n in ls:
                if n is not None:
                    if n[0] > nodes[x].x:
                        graph.adj_list[nodes[x]].append((nodes[[[no.x, no.y] for no in nodes].index(n)], 0))
                        graph.adj_list[nodes[[[no.x, no.y] for no in nodes].index(n)]].append((nodes[x], 2))
                    elif n[0] < nodes[x].x:
                        graph.adj_list[nodes[x]].append((nodes[[[no.x, no.y] for no in nodes].index(n)], 2))
                        graph.adj_list[nodes[[[no.x, no.y] for no in nodes].index(n)]].append((nodes[x], 0))
                    elif n[1] > nodes[x].y:
                        graph.adj_list[nodes[x]].append((nodes[[[no.x, no.y] for no in nodes].index(n)], 3))
                        graph.adj_list[nodes[[[no.x, no.y] for no in nodes].index(n)]].append((nodes[x], 1))
                    elif n[1] < nodes[x].y:
                        graph.adj_list[nodes[x]].append((nodes[[[no.x, no.y] for no in nodes].index(n)], 1))
                        graph.adj_list[nodes[[[no.x, no.y] for no in nodes].index(n)]].append((nodes[x], 3))
        #print([[a.x, a.y] for a in nodes])
        #print([[b.x, b.y, [(z[0].x, z[0].y, z[1]) for z in a]] for b, a in graph.adj_list.items()])
        return graph
    
        

maze = ['a#a  ###',
        '   #   #',
        ' #  b###']
botnet = MazeBotNet(maze, True)
botnet.solve()
#print([[bot.x, bot.y] for bot in botnet.bots])

2#1 2###
1 2#n  #
 #0 b###
No Solution
No Solution
No Solution
2#1 2###
1 2#n  #
 #1 b###
2#1 2###
1 2#n  #
 #1 b###
2#1 2###
1 2#2  #
 #1 b###
2#1 2###
1 2#2  #
 #1 b###


Rocky, we can transform this into a paper on cooperation within game theory and evolution. I will begin writing a blackjack program where you play against bots.

In [ ]:
# Blackjack

class Card:
    def __init__(self, suit, name, value):
        self.suit = suit
        self.name = name
        self.value = value
    def __str__(self):
        return f'{self.name} of {['Clubs', 'Hearts', 'Spades', 'Diamonds'][self.suit]}'

class Deck:
    def __init__(self):
        self.suits = [0, 1, 2, 3]
        self.names = [str(i) for i in range(2, 11)]
        self.names += ['Jack', 'King', 'Queen', 'Ace']
        self.values = [i for i in range(2, 11)]
        self.values += [10, 10, 10, 11]
        self.cards = {}
    def populate(self):    
        for i in range(4):
            for j in range(len(self.names)):
                for k in range(len(self.values)):
                    self.cards[Card(self.suits[i], self.names[j], self.values[k])] = True
class Connecter:
    def __init__(self, num_bots):
        self.bots = [Bot(self) for i in range(num_bots)]
        self.hands = {
            bot : [] for bot in self.bots
        }
        



print(Card(0, 'Ten', 10))

SyntaxError: ignored

Smple model code starts here


In [ ]:
import random
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

'''
Simpler model from yesterday:

simulate random encounters without bot net by randomly
choosing people from a set array

at the end of each day all the food collected by the hivemind will be
divided evenly between all the members

give remainder of food to random members


New model:

Have bots either work together or not work together and measure the difference
'''
'''
What happens when

selfish meets selfish: both waste food

selfish meets hive: selfish takes one, wastes other

single person: person takes both

what was the original model?

The original model didn't work so im thinking of another

also do you have discord if yes put your username and tag

I do have discord, but I will have to find my username bc I don't remember it lol
'''

self_count = 50
hive_count = 50

arr_self = np.arange(1,(self_count*2),2)
arr_hive = np.arange(2,(hive_count*2 + 1),2)
arr_all = np.concatenate((arr_self, arr_hive))

food = 100 

data_arr = np.array([]) # 2d array that store the information of everything in (3,) dimension

'''
selfish people go randomly
hive people go to predetermined food

hive_nobody = 50%
self_nobody =
hive_hive = 0% bc coordination
self_hive = 50%
self_self =
'''

class FoodClass:
    def __init__(self,id_num,person1,person2):
        self.id_num = id_num
        self.person1 = person1
        self.person2 = person2

        # person attribute:
        # 0 = nobody at food
        # 1 = selfish at food
        # 2 = hive at food

    def attributes(self):
        return np.array([self.id_num,self.person1,self.person2])

for i in range(1,food+1):
    food_obj = FoodClass(i,0,0)
    data_arr = np.append(data_arr, food_obj.attributes())
    data_arr.resize(i,3)
    
hive_food = np.sort(np.random.choice(np.arange(food) + 1 , hive_count, replace = False))
self_food = np.array([])

for i in hive_food:
    data_arr[i-1,1] = 2

for i in range(0, self_count):

    # This part could be made more efficient to reduce time complexity

    while True:
        rand_num = random.randint(0, food - 1)

        if data_arr[rand_num, 1] == 0:
            data_arr[rand_num, 1] = 1
            self_food = np.sort(np.append(self_food, rand_num+1))
            break

        elif data_arr[rand_num, 2] == 0:
            data_arr[rand_num, 2] = 1
            self_food = np.sort(np.append(self_food, rand_num+1))
            break

############################################### Unfinished ###############################################
'''
for i in range(1, food + 1):

    # big boolean expressions because im too 
    # lazy to implement anything more complicated

    if (data_arr[i-1, 1] == 1 and data_arr[i-1, 2] == 0) or (data_arr[i-1, 1] == 0 and data_arr[i-1, 2] == 1):

        # selfish person and nobody else at food, selfish takes both
    
    elif (data_arr[i-1, 1] == 2 and data_arr[i-1, 2] == 0) or (data_arr[i-1, 1] == 0 and data_arr[i-1, 2] == 2):

        # hive person and nobody elke at food, hive takes both to bring back to hive

    elif data_arr[i-1, 1] == 1 and data_arr[i-1, 2] == 1:
        
        # selfish person and another selfish person at food, both waste food
    
    elif data_arr[i-1, 1] + data_arr[i-1, 2] == 3:

        # hive person and selfish person at food, selfish takes one wastes the other
'''

'\nfor i in range(1, food + 1):\n\n    # big boolean expressions because im too \n    # lazy to implement anything more complicated\n\n    if (data_arr[i-1, 1] == 1 and data_arr[i-1, 2] == 0) or (data_arr[i-1, 1] == 0 and data_arr[i-1, 2] == 1):\n\n        # selfish person and nobody else at food, selfish takes both\n    \n    elif (data_arr[i-1, 1] == 2 and data_arr[i-1, 2] == 0) or (data_arr[i-1, 1] == 0 and data_arr[i-1, 2] == 2):\n\n        # hive person and nobody elke at food, hive takes both to bring back to hive\n\n    elif data_arr[i-1, 1] == 1 and data_arr[i-1, 2] == 1:\n        \n        # selfish person and another selfish person at food, both waste food\n    \n    elif data_arr[i-1, 1] + data_arr[i-1, 2] == 3:\n\n        # hive person and selfish person at food, selfish takes one wastes the other\n'

In [ ]:
data_arr

array([[  1.,   0.,   0.],
       [  2.,   2.,   1.],
       [  3.,   2.,   1.],
       [  4.,   2.,   0.],
       [  5.,   0.,   0.],
       [  6.,   2.,   1.],
       [  7.,   1.,   0.],
       [  8.,   2.,   1.],
       [  9.,   1.,   0.],
       [ 10.,   1.,   0.],
       [ 11.,   2.,   0.],
       [ 12.,   2.,   0.],
       [ 13.,   2.,   1.],
       [ 14.,   0.,   0.],
       [ 15.,   0.,   0.],
       [ 16.,   2.,   1.],
       [ 17.,   1.,   0.],
       [ 18.,   1.,   1.],
       [ 19.,   2.,   1.],
       [ 20.,   2.,   0.],
       [ 21.,   1.,   1.],
       [ 22.,   2.,   0.],
       [ 23.,   2.,   0.],
       [ 24.,   0.,   0.],
       [ 25.,   1.,   0.],
       [ 26.,   2.,   0.],
       [ 27.,   0.,   0.],
       [ 28.,   2.,   0.],
       [ 29.,   0.,   0.],
       [ 30.,   2.,   1.],
       [ 31.,   0.,   0.],
       [ 32.,   0.,   0.],
       [ 33.,   1.,   0.],
       [ 34.,   1.,   0.],
       [ 35.,   0.,   0.],
       [ 36.,   1.,   0.],
       [ 37.,   2.,   1.],
 